Step 1: Collect raw data.

In [4]:
import urllib.request
import datetime
import os

In [5]:
for i in range(1,28):
    url="https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean".format(i)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time_time = now.strftime("%Y%m%d%H%M%S")
    out = open(os.path.join('raw', 'NOAA_ID'+str(i)+'_'+date_and_time_time+'.csv'), 'wb')
    out.write(text)
    out.close()

Step 2: Creat dataframe.

In [6]:
import pandas as pd

In [7]:
directory = 'raw'
final_list = []
for i in range(1,28):
    new_list = [x for x in os.listdir(directory) if x.split('_')[1] == f'ID{i}']
    latest_file = sorted(new_list, key=lambda x: int(x.split('_')[2][:-4]))[-1]
    final_list.append(latest_file)




print(final_list)
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Area']


index_changer = {
    1:"Cherkasy", 2:"Chernihiv", 3:"Chernivtsi", 4:"Crimea", 5:"Dnipropetrovs'k",
    6:"Donets'k", 7:"Ivano-Frankivs'k", 8:"Kharkiv", 9:"Kherson", 10:"Khmel'nyts'kyy",
    11:"Kiev", 12:"Kiev City", 13:"Kirovohrad", 14:"Luhans'k", 15:"L'viv",
    16:"Mykolayiv", 17:"Odessa", 18:"Poltava", 19:"Rivne", 20:"Sevastopol'",
    21:"Sumy", 22:"Ternopil'", 23:"Transcarpathia", 24:"Vinnytsya", 25:"Volyn",
    26:"Zaporizhzhya", 27:"Zhytomyr",
}

['NOAA_ID1_20240722145919.csv', 'NOAA_ID2_20240722145924.csv', 'NOAA_ID3_20240722145931.csv', 'NOAA_ID4_20240722145937.csv', 'NOAA_ID5_20240722145939.csv', 'NOAA_ID6_20240722145940.csv', 'NOAA_ID7_20240722145942.csv', 'NOAA_ID8_20240722145943.csv', 'NOAA_ID9_20240722145944.csv', 'NOAA_ID10_20240722145945.csv', 'NOAA_ID11_20240722145952.csv', 'NOAA_ID12_20240722145953.csv', 'NOAA_ID13_20240722145954.csv', 'NOAA_ID14_20240722145955.csv', 'NOAA_ID15_20240722145957.csv', 'NOAA_ID16_20240722145959.csv', 'NOAA_ID17_20240722150001.csv', 'NOAA_ID18_20240722150004.csv', 'NOAA_ID19_20240722150030.csv', 'NOAA_ID20_20240722150033.csv', 'NOAA_ID21_20240722150035.csv', 'NOAA_ID22_20240722150052.csv', 'NOAA_ID23_20240722150101.csv', 'NOAA_ID24_20240722150105.csv', 'NOAA_ID25_20240722150106.csv', 'NOAA_ID26_20240722150107.csv', 'NOAA_ID27_20240722150109.csv']


In [8]:
i = 1
uncombined_df = []
for file in final_list:
    file_path = os.path.join(directory, file)
    dataframe = pd.read_csv(file_path, header = 1, names = headers)
    dataframe.drop(dataframe.index[-1], inplace=True)
    dataframe.iloc[0, 0] = dataframe.iloc[0, 0].replace('<tt><pre>', '')
    dataframe['Area'] = i
    i += 1

    dataframe = dataframe.drop(dataframe.loc[dataframe['VHI'] == -1].index)
    dataframe['Area'] = dataframe['Area'].replace({index: area for index, area in index_changer.items()})

    uncombined_df.append(dataframe)

df = pd.concat(uncombined_df, ignore_index=True)
print(df)

df.to_csv('df.csv', index=True, encoding='utf-8')

       Year  Week    SMN     SMT    VCI    TCI    VHI      Area
0      1982   1.0  0.053  260.31  45.01  39.46  42.23  Cherkasy
1      1982   2.0  0.054  262.29  46.83  31.75  39.29  Cherkasy
2      1982   3.0  0.055  263.82  48.13  27.24  37.68  Cherkasy
3      1982   4.0  0.053  265.33  46.09  23.91  35.00  Cherkasy
4      1982   5.0  0.050  265.66  41.46  26.65  34.06  Cherkasy
...     ...   ...    ...     ...    ...    ...    ...       ...
58288  2024  21.0  0.430  293.95  66.41  40.99  53.72  Zhytomyr
58289  2024  22.0  0.437  293.75  60.54  46.59  53.58  Zhytomyr
58290  2024  23.0  0.441  293.59  56.03  50.99  53.52  Zhytomyr
58291  2024  24.0  0.440  293.21  50.77  57.17  53.97  Zhytomyr
58292  2024  25.0  0.450  293.61  53.57  56.55  55.06  Zhytomyr

[58293 rows x 8 columns]
